In [214]:
import os
from importlib import reload
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import dataset_formatter
reload(dataset_formatter)
from dataset_formatter import DatasetFormatter 

In [215]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import lightning as L
import dataset
reload(dataset)
from model import BaselineModel
from dataset import EpilepsyDataset
from dataset_formatter import DatasetFormatter

In [208]:
dataset = DatasetFormatter('/workspace/data_seerpy/data_seerpy/data/')

In [ ]:
dataset.preprocess('/workspace/new_data/')

In [211]:
epilepsy_dataset = EpilepsyDataset(
    '/workspace/labels.csv', 
    '/workspace/new_data/', 
    ['Acc x', 'Acc y', 'Acc z', 'Acc Mag', 'BVP', 'EDA'],
    23040, True, True
)

In [202]:
train_size = int(0.7 * len(epilepsy_dataset))
test_size = len(epilepsy_dataset) - train_size

train_dataset, test_dataset = random_split(epilepsy_dataset, [train_size, test_size])

train_dataloader = DataLoader(epilepsy_dataset, batch_size=16, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)


In [203]:
model = BaselineModel(19)

In [204]:
trainer = L.Trainer(max_epochs=100, logger=True)
trainer.fit(model=model, train_dataloaders=train_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | rnn        | LSTM             | 472 K 
1 | fc         | Linear           | 66.0 K
2 | fc2        | Linear           | 1.0 K 
3 | sigmoid    | Sigmoid          | 0     
4 | relu       | ReLU             | 0     
5 | criterion  | CrossEntropyLoss | 0     
6 | acc_metric | BinaryAccuracy   | 0     
7 | f1_score   | BinaryF1Score    | 0     
8 | recall     | BinaryRecall     | 0     
9 | precision  | BinaryPrecision  | 0     
------------------------------------------------
539 K     Trainable params
0         Non-trainable param

Epoch 99: 100%|████████████████████████████████████████| 24/24 [00:20<00:00,  1.16it/s, v_num=56, train_loss=0.297, train_acc=0.750, train_f1=0.667, train_recall=0.500, train_precision=1.000]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|████████████████████████████████████████| 24/24 [00:20<00:00,  1.16it/s, v_num=56, train_loss=0.297, train_acc=0.750, train_f1=0.667, train_recall=0.500, train_precision=1.000]


In [205]:
trainer.test(dataloaders=test_dataloader, ckpt_path='best')

Restoring states from the checkpoint path at /workspace/team_workspace/hotinfdensedot/epilepsy_prediction/lightning_logs/version_56/checkpoints/epoch=99-step=2400.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /workspace/team_workspace/hotinfdensedot/epilepsy_prediction/lightning_logs/version_56/checkpoints/epoch=99-step=2400.ckpt


Testing DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:03<00:00,  1.94it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8482142686843872     │
│          test_f1          │    0.8422717452049255     │
│         test_loss         │    0.3162534534931183     │
│      test_precision       │    0.8187074661254883     │
│        test_recall        │    0.8857709169387817     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3162534534931183,
  'test_acc': 0.8482142686843872,
  'test_f1': 0.8422717452049255,
  'test_recall': 0.8857709169387817,
  'test_precision': 0.8187074661254883}]